In [1]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import torch


tokenizer_path = "/content/drive/MyDrive/models/qg_tokenizer"
model_path = "/content/drive/MyDrive/models/qg_model"

question_length = 512
target_length = 128
device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [3]:
device

'cuda:0'

In [7]:
def predict(context, question, ref_answer=None):
    inputs = tokenizer(
        context,
        question,
        max_length=question_length,
        padding="max_length",
        truncation=True,
        pad_to_max_length=True,
        add_special_tokens=True
    )

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(device).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(device).unsqueeze(0)
    # print(input_ids.shape)
    # print(attention_mask.shape)

    # Generate answer
    outputs = model.to(device).generate(input_ids=input_ids, attention_mask=attention_mask)

    answer = tokenizer.decode(outputs.flatten(), skip_special_tokens=True)
    if ref_answer:
        bleu = evaluate.load("google_bleu")
        bleu_score = bleu.compute(predictions=[answer], references=[ref_answer])
        print("Context: \n", context, "\n")
        print("Question: \n", question, "\n")


        return {
            "context": context,
            "reference answer": ref_answer,
            "predicted answer": answer,
            "BLEU Score: ": bleu_score,
        }
    else:
        return answer

context = "Thương nhân 1. Thương nhân bao gồm tổ chức kinh tế được thành lập hợp pháp, cá nhân hoạt động thương mại một cách độc lập, thường xuyên và có đăng ký kinh doanh. 2. Thương nhân có quyền hoạt động thương mại trong các ngành nghề, tại các địa bàn, dưới các hình thức và theo các phương thức mà pháp luật không cấm. 3. Quyền hoạt động thương mại hợp pháp của thương nhân được Nhà nước bảo hộ. 4. \
Nhà nước thực hiện độc quyền Nhà nước có thời hạn về hoạt động thương mại đối với một số hàng hóa, dịch vụ hoặc tại một số địa bàn để bảo đảm lợi ích quốc gia. Chính phủ quy định cụ thể danh mục hàng hóa, dịch vụ, địa bàn độc quyền Nhà nước."
question = "Giới thiệu về khái niệm thương nhân trong kinh tế?"

#ref answer is ground truths, labeled by human
ref_answer = "Thương nhân là cá nhân hoặc tổ chức hoạt động trong lĩnh vực kinh doanh, cung cấp sản phẩm hoặc dịch vụ để kiếm lợi nhuận"
predict(context=context, question=question, ref_answer=ref_answer)

Context: 
 Thương nhân 1. Thương nhân bao gồm tổ chức kinh tế được thành lập hợp pháp, cá nhân hoạt động thương mại một cách độc lập, thường xuyên và có đăng ký kinh doanh. 2. Thương nhân có quyền hoạt động thương mại trong các ngành nghề, tại các địa bàn, dưới các hình thức và theo các phương thức mà pháp luật không cấm. 3. Quyền hoạt động thương mại hợp pháp của thương nhân được Nhà nước bảo hộ. 4. Nhà nước thực hiện độc quyền Nhà nước có thời hạn về hoạt động thương mại đối với một số hàng hóa, dịch vụ hoặc tại một số địa bàn để bảo đảm lợi ích quốc gia. Chính phủ quy định cụ thể danh mục hàng hóa, dịch vụ, địa bàn độc quyền Nhà nước. 

Question: 
 Giới thiệu về khái niệm thương nhân trong kinh tế? 



{'context': 'Thương nhân 1. Thương nhân bao gồm tổ chức kinh tế được thành lập hợp pháp, cá nhân hoạt động thương mại một cách độc lập, thường xuyên và có đăng ký kinh doanh. 2. Thương nhân có quyền hoạt động thương mại trong các ngành nghề, tại các địa bàn, dưới các hình thức và theo các phương thức mà pháp luật không cấm. 3. Quyền hoạt động thương mại hợp pháp của thương nhân được Nhà nước bảo hộ. 4. Nhà nước thực hiện độc quyền Nhà nước có thời hạn về hoạt động thương mại đối với một số hàng hóa, dịch vụ hoặc tại một số địa bàn để bảo đảm lợi ích quốc gia. Chính phủ quy định cụ thể danh mục hàng hóa, dịch vụ, địa bàn độc quyền Nhà nước.',
 'reference answer': 'Thương nhân là cá nhân hoặc tổ chức hoạt động trong lĩnh vực kinh doanh, cung cấp sản phẩm hoặc dịch vụ để kiếm lợi nhuận',
 'predicted answer': 'Thương nhân là cá nhân hoặc tổ chức hoạt động trong lĩnh vực kinh doanh, cung cấp sản',
 'BLEU Score: ': {'google_bleu': 0.6862745098039216}}

In [ ]:
bleu = evaluate.load("google_bleu")

def evaluate_model(df, question_length):
    predictions = []
    references = []
    bleu_scores = []

    for _, row in df.iterrows():
        context = row['context']
        question = row['question']
        ref_answer = row['answer']

        inputs = tokenizer(
            context,
            question,
            max_length=question_length,
            padding="max_length",
            truncation=True,
            pad_to_max_length=True,
            add_special_tokens=True,
        )

        input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(device).unsqueeze(0)
        attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(device).unsqueeze(0)

        # Generate answer
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

        predictions.append(answer)
        references.append(ref_answer)

        # Compute BLEU score for this instance
        bleu_score = bleu.compute(predictions=[answer], references=[[ref_answer]])
        bleu_scores.append(bleu_score['score'])  # BLEU score for the current instance

    # Compute overall BLEU score
    overall_bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    return {
        "Overall BLEU Score": overall_bleu_score,
        "Individual BLEU Scores": bleu_scores
    }

# Example DataFrame
data = {
    "context": [
        "Thương nhân 1. Thương nhân bao gồm tổ chức kinh tế được thành lập hợp pháp, cá nhân hoạt động thương mại một cách độc lập, thường xuyên và có đăng ký kinh doanh.",
        "Nhà nước thực hiện độc quyền Nhà nước có thời hạn về hoạt động thương mại đối với một số hàng hóa, dịch vụ hoặc tại một số địa bàn để bảo đảm lợi ích quốc gia."
    ],
    "question": [
        "Giới thiệu về khái niệm thương nhân trong kinh tế?",
        "Nhà nước thực hiện những quyền gì trong hoạt động thương mại?"
    ],
    "answer": [
        "Thương nhân là cá nhân hoặc tổ chức hoạt động trong lĩnh vực kinh doanh, cung cấp sản phẩm hoặc dịch vụ để kiếm lợi nhuận",
        "Nhà nước thực hiện độc quyền Nhà nước có thời hạn về hoạt động thương mại đối với một số hàng hóa, dịch vụ hoặc tại một số địa bàn để bảo đảm lợi ích quốc gia."
    ]
}
df = pd.DataFrame(data)

# Evaluate the model
question_length = 128  # Adjust this value as needed
results = evaluate_model(df, question_length)

# Print overall BLEU score
print("Overall BLEU Score:", results["Overall BLEU Score"])

# Plot BLEU scores
plt.figure(figsize=(10, 5))
plt.plot(results["Individual BLEU Scores"], marker='o', linestyle='-', color='b')
plt.title('BLEU Scores for Individual Samples')
plt.xlabel('Sample Index')
plt.ylabel('BLEU Score')
plt.grid(True)
plt.show()